Shiqi Xu, June 2021, Python 3.7.4

Based on code by Sébastien in June 2018 in Python 2

## TO-DO

* interpolation cells getting index out-of-range errors

In [1]:
# -*- coding: utf-8 -*-

import os
from datetime import datetime, timedelta
import pylab as pl
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as md
from matplotlib.ticker import MaxNLocator
from scipy import stats
from math import *

## Read data

### Read MeteoData file

In [3]:
def read_meteo_file(path_meteo, file_meteo, meteo_time_shift):

    raw_meteo = open(os.path.join(path_meteo, file_meteo), 'r')
    raw_meteo_data = raw_meteo.readlines()

    meteo_time, wind_speed, wind_direction, temperature, relative_humidity, pressure = [], [], [], [], [], []

    for i in range(1, len(raw_meteo_data)):

#         date = raw_meteo_data[i].split(',')[0][:10]
#         year = int(date[:4])
#         month = int(date[5:7])
#         day = int(date[8:10])

#         time = raw_meteo_data[i].split(',')[0][-8:]
#         hour = int(time[:2])
#         minute = int(time[3:5])
#         second = int(time[6:])

#         meteo_time.append(			datetime(year, month, day, hour, minute, second) + meteo_time_shift )

        # format: "2018/08/16 01:50:51"
        date_time_meteo = datetime.strptime( raw_meteo_data[i].split(',')[0], '%Y/%m/%d %H:%M:%S' )

        meteo_time.append(			date_time_meteo + meteo_time_shift )
        wind_speed.append(			float(raw_meteo_data[i].split(',')[1]) )
        wind_direction.append(		float(raw_meteo_data[i].split(',')[2]) )
        temperature.append(			float(raw_meteo_data[i].split(',')[3]) )
        relative_humidity.append(	float(raw_meteo_data[i].split(',')[4]) )
        pressure.append(			float(raw_meteo_data[i].split(',')[5]) )
    
    return meteo_time, wind_speed, wind_direction, temperature, relative_humidity, pressure

### Read GPS file

#### From Felix's phone

In [4]:
def read_felix_gps_file(path_gps_felix, file_gps_felix):

    raw_gps_felix = open(os.path.join(path_gps_felix, file_gps_felix), 'r')
    raw_gps_felix_data = raw_gps_felix.readlines()

    gps_time_felix, gps_time_epoch_felix, lat_felix, lon_felix, alt_felix = [], [], [], [], []

    for i in range(1, len(raw_gps_felix_data)):

        if raw_gps_felix_data[i].split(',')[8] != 'network':

#             date_time = raw_gps_felix_data[i].split(',')[0]
#             year = int(date_time[:4])
#             month = int(date_time[5:7])
#             day = int(date_time[8:10])
#             hour = int(date_time[11:13])
#             minute = int(date_time[14:16])
#             second = int(date_time[17:19])
#             microsecond = int(date_time[20:23]) * 1000

#             gps_time_felix.append(			datetime(year, month, day, hour, minute, second, microsecond) )
#             gps_time_epoch_felix.append(	(datetime(year, month, day, hour, minute, second, microsecond) - datetime(1970,1,1)).total_seconds() )

            # format: "2018-08-16T01:54:53.000Z"
            date_time_gps_felix = datetime.strptime( raw_gps_felix_data[i].split(',')[0], '%Y-%m-%dT%H:%M:%S.%fZ' )

            gps_time_felix.append(			date_time_gps_felix )
            gps_time_epoch_felix.append(	(date_time_gps_felix - datetime(1970,1,1)).total_seconds() )
            lat_felix.append(				float(raw_gps_felix_data[i].split(',')[1]) )
            lon_felix.append(				float(raw_gps_felix_data[i].split(',')[2]) )
            alt_felix.append(				float(raw_gps_felix_data[i].split(',')[3]) )

    df_gps_felix = pd.DataFrame({'gps_time_felix': gps_time_felix,
                                 'gps_time_epoch_felix': gps_time_epoch_felix,
                                 'lat_felix': lat_felix,
                                 'lon_felix': lon_felix,
                                 'alt_felix': alt_felix},
                                index = gps_time_felix)

    print("df_gps_felix:")
    display(df_gps_felix)

    return gps_time_felix, gps_time_epoch_felix, lat_felix, lon_felix, alt_felix, df_gps_felix

#### From Sébastien's watch

In [5]:
def read_sebastien_gps_file(path_gps_sebastien, file_gps_sebastien):

    raw_gps_sebastien = open(os.path.join(path_gps_sebastien, file_gps_sebastien), 'r')
    raw_gps_sebastien_data = raw_gps_sebastien.readlines()

    gps_time_sebastien, gps_time_epoch_sebastien, lat_sebastien, lon_sebastien, alt_sebastien = [], [], [], [], []

    for i in range(1, len(raw_gps_sebastien_data)):

#         date_time = raw_gps_sebastien_data[i].split(',')[6]
#         year = int(date_time[:4])
#         month = int(date_time[5:7])
#         day = int(date_time[8:10])
#         hour = int(date_time[11:13])
#         minute = int(date_time[14:16])
#         second = int(date_time[17:19])

#         gps_time_sebastien.append(			datetime(year, month, day, hour, minute, second) )
#         gps_time_epoch_sebastien.append(	(datetime(year, month, day, hour, minute, second) - datetime(1970,1,1)).total_seconds() )

        # format: "2018/08/16 01:56:31+00"
        date_time_gps_sebastien = datetime.strptime( raw_gps_sebastien_data[i].split(',')[6], '%Y/%m/%d %H:%M:%S+00' )

        gps_time_sebastien.append(			date_time_gps_sebastien )
        gps_time_epoch_sebastien.append(	(date_time_gps_sebastien - datetime(1970,1,1)).total_seconds() )
        lat_sebastien.append(				float(raw_gps_sebastien_data[i].split(',')[1]) )
        lon_sebastien.append(				float(raw_gps_sebastien_data[i].split(',')[0]) )
        alt_sebastien.append(				float(raw_gps_sebastien_data[i].split(',')[5]) )

    df_gps_sebastien = pd.DataFrame({'gps_time_sebastien': gps_time_sebastien,
                                     'gps_time_epoch_sebastien': gps_time_epoch_sebastien,
                                     'lat_sebastien': lat_sebastien,
                                     'lon_sebastien': lon_sebastien,
                                     'alt_sebastien': alt_sebastien},
                                    index = gps_time_sebastien)

    print("df_gps_sebastien:")
    display(df_gps_sebastien)

    return gps_time_sebastien, gps_time_epoch_sebastien, lat_sebastien, lon_sebastien, alt_sebastien, df_gps_sebastien

### Read Picarro file

Times only, for interpolation purposes

In [6]:
def read_picarro_file(path_picarro, file_picarro, picarro_time_shift):

    raw_picarro = open(os.path.join(path_picarro, file_picarro), 'r')
    raw_picarro_data = raw_picarro.readlines()

    picarro_time, picarro_time_epoch = [], []
#     co2, co2d, co2_cal, ch4, ch4d, ch4_cal, h2o = [], [], [], [], [], [], []

    for i in range(1, len(raw_picarro_data)):

    #     date = raw_picarro_data[i].split()[0]
    #     year = int(date[-2:]) + 2000
    #     month = int(date[:2])
    #     day = int(date[3:5])

    #     time = raw_picarro_data[i].split()[1]
    #     hour = int(time[:2])
    #     minute = int(time[3:5])
    #     second = int(time[6:8])
    #     microsecond = int(time[9]) * 100000

    #     Picarro_time.append(			datetime(year, month, day, hour, minute, second, microsecond) + picarro_time_shift )
    #     Picarro_time_epoch.append(	(datetime(year, month, day, hour, minute, second, microsecond) + picarro_time_shift - datetime(1970,1,1)).total_seconds() )

        date_time_picarro_raw = raw_picarro_data[i].split()[0] + ' ' + raw_picarro_data[i].split()[1]
        # concatenated format: "08/16/18 02:12:50.3"
        date_time_picarro = datetime.strptime( date_time_picarro_raw, '%m/%d/%y %H:%M:%S.%f' )

        picarro_time.append(		date_time_picarro + picarro_time_shift)
        picarro_time_epoch.append(	(date_time_picarro + picarro_time_shift - datetime(1970,1,1)).total_seconds() )

    #     # note: need calibration script to obtain slope_CO2, intercept_CO2, slope_CH4, and intercept_CH4
    #     co2.append(					float(raw_picarro_data[i].split(',')[14]) )
    #     co2d.append(				float(raw_picarro_data[i].split(',')[15]) )
    #     co2_cal.append(				float(raw_picarro_data[i].split(',')[15]) * slope_CO2 + intercept_CO2 )
    #     ch4.append(					float(raw_picarro_data[i].split(',')[16]) )
    #     h2o.append(					float(raw_picarro_data[i].split(',')[17]) )
    #     ch4d.append(				float(raw_picarro_data[i].split(',')[16]) / (1 - 0.00982 * float(raw_picarro_data[i].split(',')[17]) - 2.393 * 10**-4 * float(raw_picarro_data[i].split(',')[17]) * float(raw_picarro_data[i].split(',')[17])) )
    #     ch4_cal.append(				float(raw_picarro_data[i].split(',')[16]) / (1 - 0.00982 * float(raw_picarro_data[i].split(',')[17]) - 2.393 * 10**-4 * float(raw_picarro_data[i].split(',')[17]) * float(raw_picarro_data[i].split(',')[17])) * slope_CH4 + intercept_CH4 )

    df_picarro_time = pd.DataFrame({'picarro_time': picarro_time,
                                    'picarro_time_epoch': picarro_time_epoch},
                                   index = picarro_time)

    print("df_picarro_time:")
    display(df_picarro_time)

    return picarro_time, picarro_time_epoch, df_picarro_time #, co2, co2d, co2_cal, ch4, ch4d, ch4_cal, h2o

## Interpolate GPS data points

### Interpolate Felix's GPS

In [7]:
def interp_felix(picarro_time, picarro_time_epoch, gps_time_epoch_felix, lat_felix, lon_felix, alt_felix):

    lat_gps_felix_interp, lon_gps_felix_interp, alt_gps_felix_interp, time_gps_felix_interp = [], [], [], []
    # ch4_gps_felix_interp, co2_gps_felix_interp, ch4_cal_gps_felix_interp, co2_cal_gps_felix_interp, h2o_gps_felix_interp = [], [], [], [], []

    # for i in range(len(picarro_time)):
    #     if (picarro_time[i] >= gps_time_felix[0]) and (picarro_time[i] <= gps_time_felix[-1]):
    #         lat_felix_interp.append(		np.interp(picarro_time_epoch[i], gps_time_epoch_felix, lat_felix) )
    #         lon_felix_interp.append(		np.interp(picarro_time_epoch[i], gps_time_epoch_felix, lon_felix) )
    #         alt_felix_interp.append(		np.interp(picarro_time_epoch[i], gps_time_epoch_felix, alt_felix) )
    #         time_felix_interp.append(		picarro_time[i] )
    # #         ch4_felix_interp.append(		ch4[i] )
    # #         co2_felix_interp.append(		co2[i] )
    # #         ch4_cal_felix_interp.append(	ch4_cal[i] )
    # #         co2_cal_felix_interp.append(	co2_cal[i] )
    # #         h2o_felix_interp.append(		h2o[i] )
    
    print("picarro_time[0]:", picarro_time[0])
    print("picarro_time[-1]:", picarro_time[-1])
    print("gps_time_epoch_felix[0]:", gps_time_epoch_felix[0])
    print("gps_time_epoch_felix[-1]:", gps_time_epoch_felix[-1])

    lat_felix_interp = np.interp(picarro_time_epoch, gps_time_epoch_felix, lat_felix)
    lon_felix_interp = np.interp(picarro_time_epoch, gps_time_epoch_felix, lon_felix)
    alt_felix_interp = np.interp(picarro_time_epoch, gps_time_epoch_felix, alt_felix)

    df_gps_felix_interp = pd.DataFrame({'picarro_time': picarro_time,
                                        'lat_felix_interp': lat_felix_interp,
                                        'lon_felix_interp': lon_felix_interp,
                                        'alt_felix_interp': alt_felix_interp},
                                       index = picarro_time)

    print("df_gps_felix_interp")
    display(df_gps_felix_interp)

    return lat_felix_interp, lon_felix_interp, alt_felix_interp, df_gps_felix_interp

### Interpolate Sébastien's GPS

In [8]:
def interp_sebastien(picarro_time, picarro_time_epoch, gps_time_epoch_sebastien, lat_sebastien, lon_sebastien, alt_sebastien):

    lat_gps_sebastien_interp, lon_gps_sebastien_interp, alt_gps_sebastien_interp, time_gps_sebastien_interp = [], [], [], []
    # ch4_gps_sebastien_interp, co2_gps_sebastien_interp, ch4_cal_gps_sebastien_interp, co2_cal_gps_sebastien_interp, h2o_gps_sebastien_interp = [], [], [], [], []

    # for i in range(len(picarro_time)):
    #     if (picarro_time[i] >= gps_time_sebastien[0]) and (picarro_time[i] <= gps_time_sebastien[-1]):
    #         lat_sebastien_interp.append(		np.interp(picarro_time_epoch[i], gps_time_epoch_sebastien, lat_sebastien) )
    #         lon_sebastien_interp.append(		np.interp(picarro_time_epoch[i], gps_time_epoch_sebastien, lon_sebastien) )
    #         alt_sebastien_interp.append(		np.interp(picarro_time_epoch[i], gps_time_epoch_sebastien, alt_sebastien) )
    #         time_sebastien_interp.append(		picarro_time[i] )
    # #         ch4_sebastien_interp.append(		ch4[i] )
    # #         co2_sebastien_interp.append(		co2[i] )
    # #         ch4_cal_sebastien_interp.append(	ch4_cal[i] )
    # #         co2_cal_sebastien_interp.append(	co2_cal[i] )
    # #         h2o_sebastien_interp.append(		h2o[i] )

    lat_sebastien_interp = np.interp(picarro_time_epoch, gps_time_epoch_sebastien, lat_sebastien)
    lon_sebastien_interp = np.interp(picarro_time_epoch, gps_time_epoch_sebastien, lon_sebastien)
    alt_sebastien_interp = np.interp(picarro_time_epoch, gps_time_epoch_sebastien, alt_sebastien)
    
    df_gps_sebastien_interp = pd.DataFrame({'picarro_time': picarro_time,
                                            'lat_sebastien_interp': lat_sebastien_interp,
                                            'lon_sebastien_interp': lon_sebastien_interp,
                                            'alt_sebastien_interp': alt_sebastien_interp},
                                           index = picarro_time)

    print("df_gps_sebastien_interp:")
    display(df_gps_sebastien_interp)

    return lat_sebastien_interp, lon_sebastien_interp, alt_sebastien_interp, df_gps_sebastien_interp

## Write to CSV files as in sync_data

### Create output directory

In [9]:
def make_output_dir(path_write_sync_raw_data):
    try:
        os.makedirs(os.path.join(path_write_sync_raw_data, 'sync_raw_data_2018'))
    except OSError:
        pass

### Write to CSV file

Creates a CSV file for one drive

In [10]:
def write_to_csv_file(path_write_sync_raw_data, drive_date, df_gps_felix_interp, df_gps_sebastien_interp):

    csv_file_name = os.path.join(path_write_sync_raw_data, 'sync_raw_data_2018', 'sync_raw_data_' + drive_date + '.csv')

    df_write_to_csv = pd.concat([df_gps_felix_interp, df_gps_sebastien_interp], axis=1)

    # remove duplicate columns
    df_write_to_csv = df_write_to_csv.loc[:,~df_write_to_csv.columns.duplicated()]

    cols = list(df_write_to_csv.columns)
#     print(cols)
#     ['picarro_time', 'lat_felix_interp', 'lon_felix_interp', 'alt_felix_interp', 'lat_sebastien_interp', 'lon_sebastien_interp', 'alt_sebastien_interp']

    cols_reorg = ['picarro_time', 'lat_felix_interp', 'lat_sebastien_interp', 'lon_felix_interp', 'lon_sebastien_interp', 'alt_felix_interp', 'alt_sebastien_interp']

    df_write_to_csv = df_write_to_csv.reindex(columns=cols_reorg)

    print("df_write_to_csv:")
    display(df_write_to_csv)
    
    df_write_to_csv.to_csv(csv_file_name, index=False)

    

    

## Main

### Set paths to raw and output data

In [13]:
path_root = os.path.dirname(os.path.dirname(os.getcwd()))

path_meteo = os.path.join(path_root, "data", "truck_campaigns_2018_raw", "Weather_Station")

path_gps_felix = os.path.join(path_root, "data", "truck_campaigns_2018_raw", "GPS", "Felix_GPS")

path_gps_sebastien = os.path.join(path_root, "data", "truck_campaigns_2018_raw", "GPS", "Sebastien_GPS")

path_picarro = os.path.join(path_root, "data", "truck_campaigns_2018_raw", "Picarro")


path_write_sync_raw_data = os.path.join(path_root, "outputs")

### Set time shifts for meteo and Picarro data

In [14]:
meteo_time_shift = timedelta(seconds=0)
# meteo_time_shift = timedelta(seconds=29)

picarro_time_shift = timedelta(seconds=20)

### Read all meteo data

In [15]:
filenames_meteo = os.listdir(path_meteo)
print("filenames_meteo:\n", filenames_meteo)

meteo_parsed_data = {}		# dict of dfs

for file in filenames_meteo:

    date_meteo = datetime(int(file[-8:-4]), int(file[-4:-2]), int(file[-2:]))

    meteo_time, wind_speed, wind_direction, temperature, relative_humidity, pressure = read_meteo_file(path_meteo, file, meteo_time_shift)

    df_meteo_day = pd.DataFrame({'meteo_time': meteo_time,
                                   'wind_speed': wind_speed,
                                   'wind_direction': wind_direction,
                                   'temperature': temperature,
                                   'relative_humidity': relative_humidity,
                                   'pressure': pressure},
                                  index = meteo_time)

    print("date_meteo:", date_meteo)
    display(df_meteo_day)

    meteo_parsed_data[date_meteo] = df_meteo_day

filenames_meteo:
 ['Meteodata_Campaign20180815', 'Meteodata_Campaign20180817', 'Meteodata_Campaign20180827', 'Meteodata_Campaign20180830', 'Meteodata_Campaign20180926', 'Meteodata_Campaign20180927', 'Meteodata_Campaign20181003', 'Meteodata_Campaign20181005', 'Meteodata_Campaign20181012']
date_meteo: 2018-08-15 00:00:00


,meteo_time,wind_speed,wind_direction,temperature,relative_humidity,pressure
2018-08-16 01:50:51,2018-08-16 01:50:51,1.5,289.0,26.3,61.0,992.4
2018-08-16 01:50:52,2018-08-16 01:50:52,1.4,299.0,26.3,61.0,992.4
2018-08-16 01:50:53,2018-08-16 01:50:53,1.4,317.0,26.3,61.0,992.4
2018-08-16 01:50:54,2018-08-16 01:50:54,1.5,315.0,26.3,61.0,992.4
2018-08-16 01:50:55,2018-08-16 01:50:55,1.5,326.0,26.3,61.0,992.4
...,...,...,...,...,...,...
2018-08-16 08:30:05,2018-08-16 08:30:05,0.4,29.0,21.0,82.0,993.5
2018-08-16 08:30:06,2018-08-16 08:30:06,0.3,53.0,21.0,82.0,993.5
2018-08-16 08:30:07,2018-08-16 08:30:07,0.3,35.0,21.0,82.0,993.5
2018-08-16 08:30:08,2018-08-16 08:30:08,0.3,35.0,21.0,82.0,993.5


date_meteo: 2018-08-17 00:00:00


,meteo_time,wind_speed,wind_direction,temperature,relative_humidity,pressure
2018-08-17 06:39:24,2018-08-17 06:39:24,1.1,213.0,23.0,86.0,991.8
2018-08-17 06:39:25,2018-08-17 06:39:25,1.2,220.0,22.9,86.0,991.8
2018-08-17 06:39:26,2018-08-17 06:39:26,1.3,217.0,22.9,86.0,991.8
2018-08-17 06:39:27,2018-08-17 06:39:27,1.1,227.0,22.9,86.0,991.8
2018-08-17 06:39:28,2018-08-17 06:39:28,1.0,224.0,22.9,86.0,991.8
...,...,...,...,...,...,...
2018-08-17 10:05:41,2018-08-17 10:05:41,1.8,165.0,22.4,95.0,990.7
2018-08-17 10:05:42,2018-08-17 10:05:42,2.3,158.0,22.4,95.0,990.7
2018-08-17 10:05:43,2018-08-17 10:05:43,2.3,159.0,22.4,95.0,990.7
2018-08-17 10:05:44,2018-08-17 10:05:44,2.3,165.0,22.4,95.0,990.7


date_meteo: 2018-08-27 00:00:00


...

,meteo_time,wind_speed,wind_direction,temperature,relative_humidity,pressure
2018-10-03 18:22:17,2018-10-03 18:22:17,0.7,142.0,16.6,66.0,995.9
2018-10-03 18:22:18,2018-10-03 18:22:18,0.8,144.0,16.6,66.0,995.9
2018-10-03 18:22:19,2018-10-03 18:22:19,1.0,138.0,16.6,66.0,995.9
2018-10-03 18:23:31,2018-10-03 18:23:31,1.0,141.0,16.4,67.0,995.9
2018-10-03 18:23:32,2018-10-03 18:23:32,1.0,146.0,16.4,67.0,995.9
...,...,...,...,...,...,...
2018-10-04 03:58:36,2018-10-04 03:58:36,0.7,139.0,15.0,93.0,988.4
2018-10-04 03:58:37,2018-10-04 03:58:37,0.8,132.0,15.0,93.0,988.4
2018-10-04 03:58:38,2018-10-04 03:58:38,0.7,135.0,15.0,93.0,988.4
2018-10-04 03:58:39,2018-10-04 03:58:39,0.7,136.0,15.0,93.0,988.4


date_meteo: 2018-10-05 00:00:00


,meteo_time,wind_speed,wind_direction,temperature,relative_humidity,pressure
2018-10-06 01:59:42,2018-10-06 01:59:42,1.2,134.0,10.3,59.0,997.6
2018-10-06 01:59:43,2018-10-06 01:59:43,1.0,138.0,10.3,59.0,997.6
2018-10-06 01:59:44,2018-10-06 01:59:44,1.0,137.0,10.3,59.0,997.6
2018-10-06 01:59:45,2018-10-06 01:59:45,0.9,140.0,10.3,59.0,997.6
2018-10-06 01:59:46,2018-10-06 01:59:46,0.8,134.0,10.3,59.0,997.5
...,...,...,...,...,...,...
2018-10-06 12:13:46,2018-10-06 12:13:46,1.0,189.0,8.3,92.0,995.3
2018-10-06 12:13:47,2018-10-06 12:13:47,1.0,189.0,8.3,92.0,995.3
2018-10-06 12:13:48,2018-10-06 12:13:48,1.1,191.0,8.3,92.0,995.3
2018-10-06 12:13:49,2018-10-06 12:13:49,1.1,194.0,8.3,92.0,995.3


date_meteo: 2018-10-12 00:00:00


,meteo_time,wind_speed,wind_direction,temperature,relative_humidity,pressure
2018-10-13 01:40:22,2018-10-13 01:40:22,0.8,216.0,7.7,66.0,987.0
2018-10-13 01:40:22,2018-10-13 01:40:22,1.1,222.0,7.7,66.0,987.0
2018-10-13 01:40:23,2018-10-13 01:40:23,1.2,217.0,7.7,66.0,987.0
2018-10-13 01:40:24,2018-10-13 01:40:24,1.1,203.0,7.7,66.0,987.0
2018-10-13 01:40:25,2018-10-13 01:40:25,1.1,199.0,7.7,66.0,987.0
...,...,...,...,...,...,...
2018-10-13 11:18:20,2018-10-13 11:18:20,1.1,277.0,5.3,85.0,987.9
2018-10-13 11:18:21,2018-10-13 11:18:21,1.2,282.0,5.3,85.0,988.0
2018-10-13 11:18:22,2018-10-13 11:18:22,1.4,284.0,5.3,85.0,988.0
2018-10-13 11:18:23,2018-10-13 11:18:23,1.5,282.0,5.3,85.0,988.0


### Read all of Felix's GPS data

In [16]:
filenames_gps_felix = os.listdir(path_gps_felix)

gps_felix_parsed_data = {}		# dict of dfs

dates_gps_felix = []

for file in filenames_gps_felix:

    if file[-4:] == '.csv':

        date_gps_felix = datetime(int(file[:4]), int(file[4:6]), int(file[6:8]))

        dates_gps_felix.append(date_gps_felix)

        gps_time_felix, gps_time_epoch_felix, lat_felix, lon_felix, alt_felix, df_gps_felix = read_felix_gps_file(path_gps_felix, file)

        gps_felix_parsed_data[date_gps_felix] = df_gps_felix

df_gps_felix:


,gps_time_felix,gps_time_epoch_felix,lat_felix,lon_felix,alt_felix
2018-08-16 01:54:53.000,2018-08-16 01:54:53.000,1.534384e+09,43.783119,-79.468935,160.197449
2018-08-16 01:56:27.000,2018-08-16 01:56:27.000,1.534385e+09,43.782993,-79.469462,124.197510
2018-08-16 01:57:52.000,2018-08-16 01:57:52.000,1.534385e+09,43.781226,-79.469196,142.616089
2018-08-16 01:59:09.000,2018-08-16 01:59:09.000,1.534385e+09,43.784467,-79.470048,171.804688
2018-08-16 02:00:53.000,2018-08-16 02:00:53.000,1.534385e+09,43.787380,-79.470252,142.083435
...,...,...,...,...,...
2018-08-16 08:23:42.000,2018-08-16 08:23:42.000,1.534408e+09,43.790397,-79.455257,152.525269
2018-08-16 08:24:44.000,2018-08-16 08:24:44.000,1.534408e+09,43.787846,-79.467201,155.052002
2018-08-16 08:26:26.443,2018-08-16 08:26:26.443,1.534408e+09,43.781514,-79.469266,133.159790
2018-08-16 08:27:29.000,2018-08-16 08:27:29.000,1.534408e+09,43.783254,-79.468814,123.408569


df_gps_felix:


,gps_time_felix,gps_time_epoch_felix,lat_felix,lon_felix,alt_felix
2018-08-16 04:00:27.000,2018-08-16 04:00:27.000,1.534392e+09,43.766869,-79.319136,219.747681
2018-08-16 04:01:52.000,2018-08-16 04:01:52.000,1.534392e+09,43.763272,-79.318360,134.302002
2018-08-16 04:02:54.000,2018-08-16 04:02:54.000,1.534392e+09,43.763314,-79.318363,149.766418
2018-08-16 04:04:29.000,2018-08-16 04:04:29.000,1.534392e+09,43.763323,-79.318315,146.789368
2018-08-16 04:05:54.000,2018-08-16 04:05:54.000,1.534392e+09,43.763325,-79.318291,142.684143
...,...,...,...,...,...
2018-08-16 08:23:42.000,2018-08-16 08:23:42.000,1.534408e+09,43.790397,-79.455257,152.525269
2018-08-16 08:24:44.000,2018-08-16 08:24:44.000,1.534408e+09,43.787846,-79.467201,155.052002
2018-08-16 08:26:26.443,2018-08-16 08:26:26.443,1.534408e+09,43.781514,-79.469266,133.159790
2018-08-16 08:27:29.000,2018-08-16 08:27:29.000,1.534408e+09,43.783254,-79.468814,123.408569


df_gps_felix:


...

,gps_time_felix,gps_time_epoch_felix,lat_felix,lon_felix,alt_felix
2018-10-13 04:00:01.391,2018-10-13 04:00:01.391,1.539403e+09,43.901552,-79.444437,224.304321
2018-10-13 04:00:04.717,2018-10-13 04:00:04.717,1.539403e+09,43.901241,-79.444321,223.642212
2018-10-13 04:00:07.974,2018-10-13 04:00:07.974,1.539403e+09,43.900906,-79.444209,221.058472
2018-10-13 04:00:11.258,2018-10-13 04:00:11.258,1.539403e+09,43.900590,-79.444147,220.452759
2018-10-13 04:00:14.540,2018-10-13 04:00:14.540,1.539403e+09,43.900269,-79.444062,216.907104
...,...,...,...,...,...
2018-10-13 11:21:15.526,2018-10-13 11:21:15.526,1.539430e+09,43.780494,-79.468367,153.262329
2018-10-13 11:21:18.803,2018-10-13 11:21:18.803,1.539430e+09,43.780479,-79.468389,155.406860
2018-10-13 11:21:22.056,2018-10-13 11:21:22.056,1.539430e+09,43.780480,-79.468393,154.182739
2018-10-13 11:21:25.396,2018-10-13 11:21:25.396,1.539430e+09,43.780485,-79.468386,158.833557


df_gps_felix:


,gps_time_felix,gps_time_epoch_felix,lat_felix,lon_felix,alt_felix
2018-11-28 21:03:01,2018-11-28 21:03:01,1.543439e+09,43.651971,-79.415506,69.715698
2018-11-28 21:03:06,2018-11-28 21:03:06,1.543439e+09,43.651911,-79.415453,47.563965
2018-11-28 21:03:13,2018-11-28 21:03:13,1.543439e+09,43.652098,-79.415620,62.479553
2018-11-28 21:03:17,2018-11-28 21:03:17,1.543439e+09,43.652230,-79.415634,67.171753
2018-11-28 21:03:24,2018-11-28 21:03:24,1.543439e+09,43.652537,-79.415681,70.793640
...,...,...,...,...,...
2018-11-29 00:33:04,2018-11-29 00:33:04,1.543452e+09,43.780732,-79.536615,163.497131
2018-11-29 00:33:07,2018-11-29 00:33:07,1.543452e+09,43.781376,-79.536724,168.310669
2018-11-29 00:33:10,2018-11-29 00:33:10,1.543452e+09,43.782059,-79.536901,168.236450
2018-11-29 00:33:13,2018-11-29 00:33:13,1.543452e+09,43.782696,-79.537074,165.949036


df_gps_felix:


,gps_time_felix,gps_time_epoch_felix,lat_felix,lon_felix,alt_felix
2018-12-04 22:46:59.000,2018-12-04 22:46:59.000,1.543964e+09,43.659061,-79.398461,68.232910
2018-12-04 22:47:04.443,2018-12-04 22:47:04.443,1.543964e+09,43.659069,-79.398498,117.749146
2018-12-04 22:47:08.000,2018-12-04 22:47:08.000,1.543964e+09,43.659109,-79.398490,103.130371
2018-12-04 22:47:11.000,2018-12-04 22:47:11.000,1.543964e+09,43.659140,-79.398458,85.842712
2018-12-04 22:47:14.000,2018-12-04 22:47:14.000,1.543964e+09,43.659144,-79.398450,81.369629
...,...,...,...,...,...
2018-12-04 23:46:45.000,2018-12-04 23:46:45.000,1.543967e+09,43.780573,-79.468265,155.305542
2018-12-04 23:46:48.000,2018-12-04 23:46:48.000,1.543967e+09,43.780573,-79.468265,155.307922
2018-12-04 23:46:51.000,2018-12-04 23:46:51.000,1.543967e+09,43.780573,-79.468265,155.326904
2018-12-04 23:46:54.000,2018-12-04 23:46:54.000,1.543967e+09,43.780573,-79.468265,155.331055


### Read all of Sébastien's GPS data

In [17]:
filenames_gps_sebastien = os.listdir(path_gps_sebastien)

gps_sebastien_parsed_data = {}

dates_gps_sebastien = []

for file in filenames_gps_sebastien:

    if file[-4:] == '.csv':

        date_gps_sebastien = datetime(int(file[-12:-8]), int(file[-8:-6]), int(file[-6:-4]))

        dates_gps_sebastien.append(date_gps_sebastien)

        gps_time_sebastien, gps_time_epoch_sebastien, lat_sebastien, lon_sebastien, alt_sebastien, df_gps_sebastien = read_sebastien_gps_file(path_gps_sebastien, file)

        gps_sebastien_parsed_data[date_gps_sebastien] = df_gps_sebastien

df_gps_sebastien:


,gps_time_sebastien,gps_time_epoch_sebastien,lat_sebastien,lon_sebastien,alt_sebastien
2018-08-16 01:56:31,2018-08-16 01:56:31,1.534385e+09,43.782782,-79.469500,172.399994
2018-08-16 01:56:32,2018-08-16 01:56:32,1.534385e+09,43.782750,-79.469496,172.199997
2018-08-16 01:56:40,2018-08-16 01:56:40,1.534385e+09,43.782586,-79.469490,172.399994
2018-08-16 01:56:42,2018-08-16 01:56:42,1.534385e+09,43.782527,-79.469490,172.199997
2018-08-16 01:56:43,2018-08-16 01:56:43,1.534385e+09,43.782492,-79.469484,172.600006
...,...,...,...,...,...
2018-08-16 08:02:37,2018-08-16 08:02:37,1.534407e+09,43.681835,-79.413172,129.600006
2018-08-16 08:02:45,2018-08-16 08:02:45,1.534407e+09,43.681838,-79.413139,129.800003
2018-08-16 08:02:47,2018-08-16 08:02:47,1.534407e+09,43.681838,-79.413139,129.800003
2018-08-16 08:02:48,2018-08-16 08:02:48,1.534407e+09,43.681838,-79.413130,129.800003


df_gps_sebastien:


,gps_time_sebastien,gps_time_epoch_sebastien,lat_sebastien,lon_sebastien,alt_sebastien
2018-08-28 04:53:18,2018-08-28 04:53:18,1.535432e+09,43.359026,-79.861079,258.799988
2018-08-28 04:53:19,2018-08-28 04:53:19,1.535432e+09,43.358916,-79.861196,268.200012
2018-08-28 04:53:20,2018-08-28 04:53:20,1.535432e+09,43.358807,-79.861314,283.000000
2018-08-28 04:53:21,2018-08-28 04:53:21,1.535432e+09,43.358714,-79.861443,294.399994
2018-08-28 04:53:23,2018-08-28 04:53:23,1.535432e+09,43.358520,-79.861724,297.000000
...,...,...,...,...,...
2018-08-28 10:05:22,2018-08-28 10:05:22,1.535451e+09,43.783101,-79.468957,245.800003
2018-08-28 10:05:57,2018-08-28 10:05:57,1.535451e+09,43.783101,-79.468953,245.800003
2018-08-28 10:06:05,2018-08-28 10:06:05,1.535451e+09,43.783101,-79.468952,245.800003
2018-08-28 10:06:14,2018-08-28 10:06:14,1.535451e+09,43.783101,-79.468952,245.800003


df_gps_sebastien:


...

,gps_time_sebastien,gps_time_epoch_sebastien,lat_sebastien,lon_sebastien,alt_sebastien
2018-10-13 02:02:16,2018-10-13 02:02:16,1.539396e+09,43.787366,-79.470655,163.600006
2018-10-13 02:02:17,2018-10-13 02:02:17,1.539396e+09,43.787370,-79.470657,161.800003
2018-10-13 02:02:19,2018-10-13 02:02:19,1.539396e+09,43.787367,-79.470658,160.399994
2018-10-13 02:02:23,2018-10-13 02:02:23,1.539396e+09,43.787448,-79.470645,161.399994
2018-10-13 02:02:26,2018-10-13 02:02:26,1.539396e+09,43.787458,-79.470739,161.800003
...,...,...,...,...,...
2018-10-13 11:51:07,2018-10-13 11:51:07,1.539431e+09,43.749073,-79.462432,151.600006
2018-10-13 11:51:08,2018-10-13 11:51:08,1.539431e+09,43.749073,-79.462432,151.600006
2018-10-13 11:51:09,2018-10-13 11:51:09,1.539431e+09,43.749073,-79.462432,151.600006
2018-10-13 11:51:14,2018-10-13 11:51:14,1.539431e+09,43.749073,-79.462430,151.600006


df_gps_sebastien:


,gps_time_sebastien,gps_time_epoch_sebastien,lat_sebastien,lon_sebastien,alt_sebastien
2018-11-28 21:05:02,2018-11-28 21:05:02,1.543439e+09,43.655053,-79.416697,93.400002
2018-11-28 21:05:04,2018-11-28 21:05:04,1.543439e+09,43.655063,-79.416697,93.400002
2018-11-28 21:05:12,2018-11-28 21:05:12,1.543439e+09,43.655090,-79.416712,93.599998
2018-11-28 21:05:14,2018-11-28 21:05:14,1.543439e+09,43.655112,-79.416748,93.800003
2018-11-28 21:05:22,2018-11-28 21:05:22,1.543439e+09,43.655247,-79.417543,93.800003
...,...,...,...,...,...
2018-11-29 02:12:51,2018-11-29 02:12:51,1.543458e+09,43.780592,-79.468726,149.199997
2018-11-29 02:12:58,2018-11-29 02:12:58,1.543458e+09,43.780472,-79.468544,148.000000
2018-11-29 02:13:06,2018-11-29 02:13:06,1.543458e+09,43.780448,-79.468292,147.199997
2018-11-29 02:13:09,2018-11-29 02:13:09,1.543458e+09,43.780488,-79.468269,147.000000


df_gps_sebastien:


,gps_time_sebastien,gps_time_epoch_sebastien,lat_sebastien,lon_sebastien,alt_sebastien
2018-12-04 22:47:18,2018-12-04 22:47:18,1.543964e+09,43.659097,-79.398308,111.000000
2018-12-04 22:47:19,2018-12-04 22:47:19,1.543964e+09,43.659107,-79.398305,111.000000
2018-12-04 22:47:42,2018-12-04 22:47:42,1.543964e+09,43.659062,-79.398407,111.199997
2018-12-04 22:47:48,2018-12-04 22:47:48,1.543964e+09,43.659083,-79.398415,111.199997
2018-12-04 22:47:52,2018-12-04 22:47:52,1.543964e+09,43.659140,-79.398444,111.199997
...,...,...,...,...,...
2018-12-05 05:36:35,2018-12-05 05:36:35,1.543988e+09,43.780542,-79.468285,200.800003
2018-12-05 05:36:37,2018-12-05 05:36:37,1.543988e+09,43.780543,-79.468277,200.800003
2018-12-05 05:36:39,2018-12-05 05:36:39,1.543988e+09,43.780541,-79.468277,200.800003
2018-12-05 05:36:41,2018-12-05 05:36:41,1.543988e+09,43.780541,-79.468277,200.800003


### Read all Picarro timestamps

In [19]:
# from comments in Sebastien's code

files_picarro = {'2018-08-15': os.path.join('20180816', 'CFADS59-20180816-0148-Data.dat'),
                 '2018-08-16': os.path.join('20180817', 'CFADS59-20180817-0627-Data.dat'),
                 '2018-08-27': os.path.join('20180828', 'CFADS59-20180828-0317-Data.dat'),
                 '2018-08-30': os.path.join('20180830', 'CFADS59-20180830-1436-Data.dat'),
                 '2018-09-26': os.path.join('20180926', 'CFADS59-20180926-Data.dat'),
                 '2018-10-03': os.path.join('20181003', 'CFADS59-20181003-Data.dat'),
                 '2018-10-05': os.path.join('20181006', 'CFADS59-20181006-0156-Data.dat'),
                 '2018-10-12': os.path.join('20181013', 'CFADS59-20181013-Data.dat')}

dates_picarro = list(files_picarro.keys())

In [20]:
picarro_parsed_data = {}

for date in files_picarro:

    picarro_time, picarro_time_epoch, df_picarro_time = read_picarro_file(path_picarro, files_picarro[date], picarro_time_shift)

    picarro_parsed_data[date] = df_picarro_time

df_picarro_time:


,picarro_time,picarro_time_epoch
2018-08-16 02:13:10.300,2018-08-16 02:13:10.300,1.534386e+09
2018-08-16 02:13:11.400,2018-08-16 02:13:11.400,1.534386e+09
2018-08-16 02:13:14.400,2018-08-16 02:13:14.400,1.534386e+09
2018-08-16 02:13:15.600,2018-08-16 02:13:15.600,1.534386e+09
2018-08-16 02:13:18.400,2018-08-16 02:13:18.400,1.534386e+09
...,...,...
2018-08-16 08:30:11.100,2018-08-16 08:30:11.100,1.534408e+09
2018-08-16 08:30:14.100,2018-08-16 08:30:14.100,1.534408e+09
2018-08-16 08:30:15.200,2018-08-16 08:30:15.200,1.534408e+09
2018-08-16 08:30:18.200,2018-08-16 08:30:18.200,1.534408e+09


df_picarro_time:


,picarro_time,picarro_time_epoch
2018-08-17 06:51:11.900,2018-08-17 06:51:11.900,1.534489e+09
2018-08-17 06:51:13.100,2018-08-17 06:51:13.100,1.534489e+09
2018-08-17 06:51:15.800,2018-08-17 06:51:15.800,1.534489e+09
2018-08-17 06:51:17.000,2018-08-17 06:51:17.000,1.534489e+09
2018-08-17 06:51:19.700,2018-08-17 06:51:19.700,1.534489e+09
...,...,...
2018-08-17 10:23:47.600,2018-08-17 10:23:47.600,1.534501e+09
2018-08-17 10:23:50.600,2018-08-17 10:23:50.600,1.534501e+09
2018-08-17 10:23:51.800,2018-08-17 10:23:51.800,1.534501e+09
2018-08-17 10:23:54.800,2018-08-17 10:23:54.800,1.534501e+09


df_picarro_time:


...

,picarro_time,picarro_time_epoch
2018-10-03 18:38:46.600,2018-10-03 18:38:46.600,1.538592e+09
2018-10-03 18:38:47.800,2018-10-03 18:38:47.800,1.538592e+09
2018-10-03 18:38:50.600,2018-10-03 18:38:50.600,1.538592e+09
2018-10-03 18:38:51.800,2018-10-03 18:38:51.800,1.538592e+09
2018-10-03 18:38:54.600,2018-10-03 18:38:54.600,1.538592e+09
...,...,...
2018-10-04 03:57:55.100,2018-10-04 03:57:55.100,1.538625e+09
2018-10-04 03:57:57.900,2018-10-04 03:57:57.900,1.538625e+09
2018-10-04 03:57:59.100,2018-10-04 03:57:59.100,1.538625e+09
2018-10-04 03:58:01.900,2018-10-04 03:58:01.900,1.538625e+09


df_picarro_time:


,picarro_time,picarro_time_epoch
2018-10-06 02:32:26.500,2018-10-06 02:32:26.500,1.538793e+09
2018-10-06 02:32:27.600,2018-10-06 02:32:27.600,1.538793e+09
2018-10-06 02:32:30.800,2018-10-06 02:32:30.800,1.538793e+09
2018-10-06 02:32:31.900,2018-10-06 02:32:31.900,1.538793e+09
2018-10-06 02:32:34.900,2018-10-06 02:32:34.900,1.538793e+09
...,...,...
2018-10-06 12:16:57.600,2018-10-06 12:16:57.600,1.538828e+09
2018-10-06 12:17:00.900,2018-10-06 12:17:00.900,1.538828e+09
2018-10-06 12:17:02.100,2018-10-06 12:17:02.100,1.538828e+09
2018-10-06 12:17:05.200,2018-10-06 12:17:05.200,1.538828e+09


df_picarro_time:


,picarro_time,picarro_time_epoch
2018-10-13 03:10:01.300,2018-10-13 03:10:01.300,1.539400e+09
2018-10-13 03:10:02.500,2018-10-13 03:10:02.500,1.539400e+09
2018-10-13 03:10:05.800,2018-10-13 03:10:05.800,1.539400e+09
2018-10-13 03:10:07.100,2018-10-13 03:10:07.100,1.539400e+09
2018-10-13 03:10:10.300,2018-10-13 03:10:10.300,1.539400e+09
...,...,...
2018-10-13 11:16:32.000,2018-10-13 11:16:32.000,1.539429e+09
2018-10-13 11:16:34.800,2018-10-13 11:16:34.800,1.539429e+09
2018-10-13 11:16:36.000,2018-10-13 11:16:36.000,1.539429e+09
2018-10-13 11:16:38.700,2018-10-13 11:16:38.700,1.539429e+09


### Interpolate all of Felix's GPS data

In [21]:
for i in range(len(dates_gps_felix)):

    picarro_time = picarro_parsed_data[dates_picarro[i]].loc[:, "picarro_time"]
    picarro_time_epoch = picarro_parsed_data[dates_picarro[i]].loc[:, "picarro_time_epoch"]
    gps_time_epoch_felix = gps_felix_parsed_data[dates_gps_felix[i]].loc[:, "gps_time_epoch_felix"]
    lat_felix = gps_felix_parsed_data[dates_gps_felix[i]].loc[:, "lat_felix"]
    lon_felix = gps_felix_parsed_data[dates_gps_felix[i]].loc[:, "lon_felix"]
    alt_felix = gps_felix_parsed_data[dates_gps_felix[i]].loc[:, "alt_felix"]

    lat_felix_interp, lon_felix_interp, alt_felix_interp, df_gps_felix_interp = interp_felix(picarro_time, picarro_time_epoch, gps_time_epoch_felix, lat_felix, lon_felix, alt_felix)

picarro_time[0]: 2018-08-16 02:13:10.300000
picarro_time[-1]: 2018-08-16 08:30:19.300000
gps_time_epoch_felix[0]: 1534384493.0
gps_time_epoch_felix[-1]: 1534408152.0
df_gps_felix_interp


,picarro_time,lat_felix_interp,lon_felix_interp,alt_felix_interp
picarro_time,,,,
2018-08-16 02:13:10.300,2018-08-16 02:13:10.300,43.792941,-79.473678,136.333239
2018-08-16 02:13:11.400,2018-08-16 02:13:11.400,43.792947,-79.473695,136.637168
2018-08-16 02:13:14.400,2018-08-16 02:13:14.400,43.792963,-79.473742,137.466067
2018-08-16 02:13:15.600,2018-08-16 02:13:15.600,43.792969,-79.473761,137.797626
2018-08-16 02:13:18.400,2018-08-16 02:13:18.400,43.792984,-79.473805,138.571264
...,...,...,...,...
2018-08-16 08:30:11.100,2018-08-16 08:30:11.100,43.783175,-79.469029,133.424866
2018-08-16 08:30:14.100,2018-08-16 08:30:14.100,43.783175,-79.469029,133.424866
2018-08-16 08:30:15.200,2018-08-16 08:30:15.200,43.783175,-79.469029,133.424866


picarro_time[0]: 2018-08-17 06:51:11.900000
picarro_time[-1]: 2018-08-17 10:23:56
gps_time_epoch_felix[0]: 1534392027.0
gps_time_epoch_felix[-1]: 1534408152.0
df_gps_felix_interp


,picarro_time,lat_felix_interp,lon_felix_interp,alt_felix_interp
picarro_time,,,,
2018-08-17 06:51:11.900,2018-08-17 06:51:11.900,43.783175,-79.469029,133.424866
2018-08-17 06:51:13.100,2018-08-17 06:51:13.100,43.783175,-79.469029,133.424866
2018-08-17 06:51:15.800,2018-08-17 06:51:15.800,43.783175,-79.469029,133.424866
2018-08-17 06:51:17.000,2018-08-17 06:51:17.000,43.783175,-79.469029,133.424866
2018-08-17 06:51:19.700,2018-08-17 06:51:19.700,43.783175,-79.469029,133.424866
...,...,...,...,...
2018-08-17 10:23:47.600,2018-08-17 10:23:47.600,43.783175,-79.469029,133.424866
2018-08-17 10:23:50.600,2018-08-17 10:23:50.600,43.783175,-79.469029,133.424866
2018-08-17 10:23:51.800,2018-08-17 10:23:51.800,43.783175,-79.469029,133.424866


picarro_time[0]: 2018-08-28 03:40:19.400000
picarro_time[-1]: 2018-08-28 10:06:06.600000
gps_time_epoch_felix[0]: 1534488324.606
gps_time_epoch_felix[-1]: 1534500415.63
df_gps_felix_interp


...

picarro_time[0]: 2018-10-06 02:32:26.500000
picarro_time[-1]: 2018-10-06 12:17:06.500000
gps_time_epoch_felix[0]: 1538014404.0
gps_time_epoch_felix[-1]: 1538044177.633
df_gps_felix_interp


,picarro_time,lat_felix_interp,lon_felix_interp,alt_felix_interp
picarro_time,,,,
2018-10-06 02:32:26.500,2018-10-06 02:32:26.500,43.783032,-79.468911,153.907227
2018-10-06 02:32:27.600,2018-10-06 02:32:27.600,43.783032,-79.468911,153.907227
2018-10-06 02:32:30.800,2018-10-06 02:32:30.800,43.783032,-79.468911,153.907227
2018-10-06 02:32:31.900,2018-10-06 02:32:31.900,43.783032,-79.468911,153.907227
2018-10-06 02:32:34.900,2018-10-06 02:32:34.900,43.783032,-79.468911,153.907227
...,...,...,...,...
2018-10-06 12:16:57.600,2018-10-06 12:16:57.600,43.783032,-79.468911,153.907227
2018-10-06 12:17:00.900,2018-10-06 12:17:00.900,43.783032,-79.468911,153.907227
2018-10-06 12:17:02.100,2018-10-06 12:17:02.100,43.783032,-79.468911,153.907227


picarro_time[0]: 2018-10-13 03:10:01.300000
picarro_time[-1]: 2018-10-13 11:16:40
gps_time_epoch_felix[0]: 1538020802.643
gps_time_epoch_felix[-1]: 1538044177.633
df_gps_felix_interp


,picarro_time,lat_felix_interp,lon_felix_interp,alt_felix_interp
picarro_time,,,,
2018-10-13 03:10:01.300,2018-10-13 03:10:01.300,43.783032,-79.468911,153.907227
2018-10-13 03:10:02.500,2018-10-13 03:10:02.500,43.783032,-79.468911,153.907227
2018-10-13 03:10:05.800,2018-10-13 03:10:05.800,43.783032,-79.468911,153.907227
2018-10-13 03:10:07.100,2018-10-13 03:10:07.100,43.783032,-79.468911,153.907227
2018-10-13 03:10:10.300,2018-10-13 03:10:10.300,43.783032,-79.468911,153.907227
...,...,...,...,...
2018-10-13 11:16:32.000,2018-10-13 11:16:32.000,43.783032,-79.468911,153.907227
2018-10-13 11:16:34.800,2018-10-13 11:16:34.800,43.783032,-79.468911,153.907227
2018-10-13 11:16:36.000,2018-10-13 11:16:36.000,43.783032,-79.468911,153.907227


IndexError: list index out of range

### Interpolate all of Sébastien's GPS data

In [27]:
for i in range(len(dates_gps_sebastien)):

    picarro_time = picarro_parsed_data[dates_picarro[i]].loc[:, "picarro_time"]
    picarro_time_epoch = picarro_parsed_data[dates_picarro[i]].loc[:, "picarro_time_epoch"]
    gps_time_epoch_sebastien = gps_sebastien_parsed_data[dates_gps_sebastien[i]].loc[:, "gps_time_epoch_sebastien"]
    lat_sebastien = gps_sebastien_parsed_data[dates_gps_sebastien[i]].loc[:, "lat_sebastien"]
    lon_sebastien = gps_sebastien_parsed_data[dates_gps_sebastien[i]].loc[:, "lon_sebastien"]
    alt_sebastien = gps_sebastien_parsed_data[dates_gps_sebastien[i]].loc[:, "alt_sebastien"]

    lat_sebastien_interp, lon_sebastien_interp, alt_sebastien_interp, df_gps_sebastien_interp = interp_sebastien(picarro_time, picarro_time_epoch, gps_time_epoch_sebastien, lat_sebastien, lon_sebastien, alt_sebastien)

df_gps_sebastien_interp:


,picarro_time,lat_sebastien_interp,lon_sebastien_interp,alt_sebastien_interp
picarro_time,,,,
2018-08-16 02:13:10.300,2018-08-16 02:13:10.300,43.792482,-79.473213,168.600006
2018-08-16 02:13:11.400,2018-08-16 02:13:11.400,43.792482,-79.473236,168.600006
2018-08-16 02:13:14.400,2018-08-16 02:13:14.400,43.792482,-79.473299,168.600006
2018-08-16 02:13:15.600,2018-08-16 02:13:15.600,43.792482,-79.473324,168.600006
2018-08-16 02:13:18.400,2018-08-16 02:13:18.400,43.792482,-79.473382,168.600006
...,...,...,...,...
2018-08-16 08:30:11.100,2018-08-16 08:30:11.100,43.681857,-79.413047,130.000000
2018-08-16 08:30:14.100,2018-08-16 08:30:14.100,43.681857,-79.413047,130.000000
2018-08-16 08:30:15.200,2018-08-16 08:30:15.200,43.681857,-79.413047,130.000000


df_gps_sebastien_interp:


,picarro_time,lat_sebastien_interp,lon_sebastien_interp,alt_sebastien_interp
picarro_time,,,,
2018-08-17 06:51:11.900,2018-08-17 06:51:11.900,43.359026,-79.861079,258.799988
2018-08-17 06:51:13.100,2018-08-17 06:51:13.100,43.359026,-79.861079,258.799988
2018-08-17 06:51:15.800,2018-08-17 06:51:15.800,43.359026,-79.861079,258.799988
2018-08-17 06:51:17.000,2018-08-17 06:51:17.000,43.359026,-79.861079,258.799988
2018-08-17 06:51:19.700,2018-08-17 06:51:19.700,43.359026,-79.861079,258.799988
...,...,...,...,...
2018-08-17 10:23:47.600,2018-08-17 10:23:47.600,43.359026,-79.861079,258.799988
2018-08-17 10:23:50.600,2018-08-17 10:23:50.600,43.359026,-79.861079,258.799988
2018-08-17 10:23:51.800,2018-08-17 10:23:51.800,43.359026,-79.861079,258.799988


df_gps_sebastien_interp:


...

df_gps_sebastien_interp:


,picarro_time,lat_sebastien_interp,lon_sebastien_interp,alt_sebastien_interp
picarro_time,,,,
2018-10-06 02:32:26.500,2018-10-06 02:32:26.500,43.787366,-79.470655,163.600006
2018-10-06 02:32:27.600,2018-10-06 02:32:27.600,43.787366,-79.470655,163.600006
2018-10-06 02:32:30.800,2018-10-06 02:32:30.800,43.787366,-79.470655,163.600006
2018-10-06 02:32:31.900,2018-10-06 02:32:31.900,43.787366,-79.470655,163.600006
2018-10-06 02:32:34.900,2018-10-06 02:32:34.900,43.787366,-79.470655,163.600006
...,...,...,...,...
2018-10-06 12:16:57.600,2018-10-06 12:16:57.600,43.787366,-79.470655,163.600006
2018-10-06 12:17:00.900,2018-10-06 12:17:00.900,43.787366,-79.470655,163.600006
2018-10-06 12:17:02.100,2018-10-06 12:17:02.100,43.787366,-79.470655,163.600006


df_gps_sebastien_interp:


,picarro_time,lat_sebastien_interp,lon_sebastien_interp,alt_sebastien_interp
picarro_time,,,,
2018-10-13 03:10:01.300,2018-10-13 03:10:01.300,43.655053,-79.416697,93.400002
2018-10-13 03:10:02.500,2018-10-13 03:10:02.500,43.655053,-79.416697,93.400002
2018-10-13 03:10:05.800,2018-10-13 03:10:05.800,43.655053,-79.416697,93.400002
2018-10-13 03:10:07.100,2018-10-13 03:10:07.100,43.655053,-79.416697,93.400002
2018-10-13 03:10:10.300,2018-10-13 03:10:10.300,43.655053,-79.416697,93.400002
...,...,...,...,...
2018-10-13 11:16:32.000,2018-10-13 11:16:32.000,43.655053,-79.416697,93.400002
2018-10-13 11:16:34.800,2018-10-13 11:16:34.800,43.655053,-79.416697,93.400002
2018-10-13 11:16:36.000,2018-10-13 11:16:36.000,43.655053,-79.416697,93.400002


IndexError: list index out of range

### Write weather station data and interpolated GPS data to CSV files

In [21]:
make_output_dir(path_write_sync_raw_data)

for date in files_picarro:

    write_to_csv_file(path_write_sync_raw_data, date, df_gps_felix_interp, df_gps_sebastien_interp)

NameError: name 'df_gps_sebastien_interp' is not defined